# PaddlePaddle BYOS

## Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local` (or `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance).

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).

**Note, you can only run a single local notebook at one time.**

In [1]:
# !/bin/bash ./utils/setup.sh

The user has root access.
nvidia-docker2 already installed. We are good to go!
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to create a convolutional neural network model to train the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) using **PyTorch in local mode**.

### Set up the environment

This notebook was created and tested on a single ml.p2.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [2]:
import os
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-PaddleNLP'

role = sagemaker.get_execution_role()

In [3]:
# import subprocess

# instance_type = 'local'

# if subprocess.call('nvidia-smi') == 0:
#     ## Set type to GPU if one is present
#     instance_type = 'local_gpu'
    
# print("Instance type = " + instance_type)

Instance type = local_gpu


### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [6]:
# base_dir = 'file:///home/ec2-user/SageMaker/paddlenlp_sagemaker/data/'
# inputs = {'training': base_dir}
# print(inputs)

{'training': 'file:///home/ec2-user/SageMaker/paddlenlp_sagemaker/data/'}


## Script Functions

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the model_fn() is called to determine how to load your trained model. The model_fn() along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* model_fn(model_dir) - loads your model.
* input_fn(serialized_input_data, content_type) - deserializes predictions to predict_fn.
* output_fn(prediction_output, accept) - serializes predictions from predict_fn.
* predict_fn(input_data, model) - calls a model on data deserialized in input_fn.

The model_fn() is the only function that doesn't have a default implementation and is required by the user for using PyTorch on SageMaker. 

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


In [14]:
# from sagemaker.pytorch import PyTorch

# # git_config = {'repo': 'https://github.com/PaddlePaddle/PaddleNLP.git', 'branch': 'develop'}

# hyperparameters = {'train_path': '/opt/ml/input/data/training/train.txt', 
#                    'dev_path': '/opt/ml/input/data/training/dev.txt', 
#                    'save_dir': '/opt/ml/model', 
#                    'learning_rate': 1e-5, 
#                    'batch_size': 16, 
#                    'max_seq_len':512, 
#                    'num_epochs': 100, 
#                    'model': 'uie-base',
#                    'seed': 1000,
#                    'logging_steps': 10,
#                    'valid_steps': 100,
#                    'device': 'gpu'}

# estimator = PyTorch(entry_point='finetune.py',
#                             source_dir='./',
# #                             source_dir='model_zoo/uie/',
# #                             git_config=git_config,
#                             role=role,
#                             hyperparameters=hyperparameters,
#                             framework_version='1.9.1',
#                             py_version='py38',
#                             script_mode=True,
#                             instance_count=1,  # 1 or 2 or ...
#                             instance_type=instance_type)

# estimator.fit(inputs)

Creating czv6o4bv3p-algo-1-tuox4 ... 
Creating czv6o4bv3p-algo-1-tuox4 ... done
Attaching to czv6o4bv3p-algo-1-tuox4
czv6o4bv3p-algo-1-tuox4 | 2022-06-07 02:05:27,102 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
czv6o4bv3p-algo-1-tuox4 | 2022-06-07 02:05:27,125 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
czv6o4bv3p-algo-1-tuox4 | 2022-06-07 02:05:27,128 sagemaker_pytorch_container.training INFO     Invoking user training script.
czv6o4bv3p-algo-1-tuox4 | 2022-06-07 02:05:27,279 sagemaker-training-toolkit ERROR    Reporting training FAILURE
czv6o4bv3p-algo-1-tuox4 | 2022-06-07 02:05:27,279 sagemaker-training-toolkit ERROR    Framework Error: 
czv6o4bv3p-algo-1-tuox4 | Traceback (most recent call last):
czv6o4bv3p-algo-1-tuox4 |   File "/opt/conda/lib/python3.8/site-packages/sagemaker_training/trainer.py", line 87, in train
czv6o4bv3p-algo-1-tuox4 |     entrypoint()
czv6o4bv3p-algo-1-tuox4 |   File

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmp41_wj6ql/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

## SageMaker Training using GPU instance

In [15]:
WORK_DIRECTORY = '/home/ec2-user/SageMaker/paddlenlp_sagemaker/data/'

data_location = sagemaker_session.upload_data(WORK_DIRECTORY, key_prefix=prefix)

inputs = {'training': data_location}

print(inputs)

{'training': 's3://sagemaker-us-east-1-579019700964/sagemaker/DEMO-PaddleNLP'}


In [ ]:
from sagemaker.pytorch import PyTorch

hyperparameters = {'train_path': '/opt/ml/input/data/training/train.txt', 
                   'dev_path': '/opt/ml/input/data/training/dev.txt', 
                   'save_dir': '/opt/ml/model', 
                   'learning_rate': 1e-5, 
                   'batch_size': 16, 
                   'max_seq_len':512, 
                   'num_epochs': 100, 
                   'model': 'uie-base',
                   'seed': 1000,
                   'logging_steps': 10,
                   'valid_steps': 100,
                   'device': 'gpu'}

instance_type = 'ml.p3.2xlarge'  # 'ml.p3.2xlarge' or 'ml.p3.8xlarge' or ...

# git_config = {'repo': 'https://github.com/PaddlePaddle/PaddleNLP.git', 'branch': 'develop'}

estimator = PyTorch(entry_point='finetune.py',
#                             source_dir='model_zoo/uie/',
#                             git_config=git_config,
                            role=role,
                            hyperparameters=hyperparameters,
                            framework_version='1.9.1',
                            py_version='py38',
                            script_mode=True,
                            instance_count=1,  # 1 or 2 or ...
                            instance_type=instance_type)

estimator.fit(inputs)

2022-06-07 03:27:58 Starting - Starting the training job...
2022-06-07 03:28:26 Starting - Preparing the instances for trainingProfilerReport-1654572478: InProgress
.........
2022-06-07 03:29:58 Downloading - Downloading input data
2022-06-07 03:29:58 Training - Downloading the training image...................

In [ ]:
training_job_name = estimator.latest_training_job.name
# training_job_name = 'pytorch-training-2022-03-03-04-38-25-840'
print(training_job_name)

# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint (in this case locally) which serves prediction requests in real-time.

In [ ]:
!rm -rf model.tar.gz
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz .
!tar -xvf model.tar.gz

In [ ]:
!cp tutorial/weights/best.pt model/
!cd model && tar -czvf ../model-pytorch.tar.gz *

!aws s3 cp model-pytorch.tar.gz s3://$bucket/$training_job_name/output/model-pytorch.tar.gz

In [ ]:
# instance_type = 'local'
instance_type = 'ml.m5.xlarge'

# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-pytorch.tar.gz'.format(bucket, training_job_name), role=role,
                             entry_point='infer.py', framework_version='1.9.1', py_version='py38')

predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

# Invoking the endpoint

In [ ]:
texts = ['"北京市海淀区人民法院\n民事判决书\n(199x)建初字第xxx号\n原告：张三。\n委托代理人李四，北京市 A律师事务所律师。\n被告：B公司，法定代表人王五，开发公司总经理。\n委托代理人赵六，北京市 C律师事务所律师。"', 
         '原告赵六，2022年5月29日生\n委托代理人孙七，深圳市C律师事务所律师。\n被告周八，1990年7月28日出生\n委托代理人吴九，山东D律师事务所律师']

outputs = predictor.predict(texts)
print('outputs: ', outputs)

# Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [ ]:
# estimator.delete_endpoint()
predictor.delete_endpoint()